In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

# Load data SQL

In [2]:
def load_sql(query: str) -> pd.DataFrame:
    load_dotenv()
    db_user= os.getenv("DB_USER")
    db_password= os.getenv("DB_PASSWORD")
    db_host= os.getenv("DB_HOST")
    db_name= os.getenv("DB_NAME")
    engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}/{db_name}')
    with engine.connect() as conn:
        df = pd.read_sql_query(text(query), conn)
        return df

df = load_sql("SELECT * FROM raw.pharmacy_sales;")
df = df.sort_values(by=["distributor", "product_name", "year", "city", "month"])
df.head()

,distributor,customer_name,city,country,latitude,longitude,channel,sub_channel,product_name,product_class,quantity,price,sales,month,year,sales_rep_name,manager,sales_team
207687,Bashirian-Kassulke,Rogahn-Klein Pharma Plc,Leinfelden-Echterdingen,Germany,48.6928,9.1428,Pharmacy,Institution,Abatatriptan,Antibiotics,2.0,742.0,1484.0,February,2020,Stella Given,Alisha Cordwell,Charlie
187350,Bashirian-Kassulke,Runolfsson-Halvorson Pharm,Rheinberg,Germany,51.5467,6.6006,Pharmacy,Retail,Abranatal Lysoprosate,Antiseptics,15826.0,681.0,10777506.0,August,2019,Mary Gerrard,Britanny Bold,Delta
254078,Bashirian-Kassulke,Hane Ltd Pharmaceutical Ltd,Aichach,Germany,48.4500,11.1333,Hospital,Private,Abranatal Lysoprosate,Antiseptics,432.0,681.0,294192.0,December,2020,Anne Wu,Britanny Bold,Delta
175417,Bashirian-Kassulke,Doyle-Tillman Pharmaceutical Limited,Zirndorf,Germany,49.4500,10.9500,Pharmacy,Institution,Acantaine,Antibiotics,50.0,66.0,3300.0,June,2019,Thompson Crawford,James Goodwill,Alfa
246485,Bashirian-Kassulke,"Langworth, Olson and Satterfield Pharmacy",Meschede,Germany,51.3503,8.2836,Hospital,Government,Aciprex,Antipiretics,150.0,421.0,63150.0,November,2020,Thompson Crawford,James Goodwill,Alfa


In [3]:
df[['quantity', 'sales']].head(10)

,quantity,sales
207687,2.0,1484.0
187350,15826.0,10777506.0
254078,432.0,294192.0
175417,50.0,3300.0
246485,150.0,63150.0
232401,20.0,8420.0
53000,2500.0,1695000.0
254079,320.0,216960.0
158400,60.0,1440.0
188559,2000.0,48000.0


# Feature Engineering

In [4]:
features = (df.groupby(["distributor",
                    "channel",
                    "sub_channel",
                    "city",
                    "product_name",
                    "product_class",
                    "sales_team",
                    "year",
                    "month",]).agg(
                total_quantity=("quantity", "sum"),
                total_sales=("sales", "sum"),
                avg_price=("price", "mean"),
                    ).reset_index())

sort_columns = ["distributor", "channel", "sub_channel", "city", 
    "product_name", "product_class", "sales_team", 
    "year", "month"]
features = features.sort_values(by=sort_columns)

In [ ]:
grp_cols = [
    "distributor", # Concerned columns to identify what makes features works
] # time series not included
grp = features.groupby(grp_cols)

# add rolling features
features["rolling_avg_sales_3m"] = grp["total_sales"].transform(
    lambda x: x.rolling(window=3, min_periods=1).mean()
)
features["sales_growth_pct"] = grp["total_sales"].transform(
    lambda x: x.pct_change() * 100
)

In [6]:
# Clean Nan and Inf values
features_cleaned = features.replace([np.inf, -np.inf], np.nan).fillna(0)
features_cleaned.head(10)

,distributor,channel,sub_channel,city,product_name,product_class,sales_team,year,month,total_quantity,total_sales,avg_price,rolling_avg_sales_3m,sales_growth_pct
0,Bashirian-Kassulke,Hospital,Government,Altenburg,Symbitrim,Analgesics,Bravo,2019,August,29400.0,15758400.0,536.0,1.575840e+07,0.000000
1,Bashirian-Kassulke,Hospital,Government,Bad Salzuflen,Adrecetam Barazoxane,Antimalarial,Bravo,2020,December,16.0,384.0,24.0,7.879392e+06,-99.997563
2,Bashirian-Kassulke,Hospital,Government,Bad Tölz,Albuterenone,Antimalarial,Bravo,2019,November,2000.0,164000.0,82.0,5.307595e+06,42608.333333
3,Bashirian-Kassulke,Hospital,Government,Bergkamen,Choriogestrel,Antiseptics,Bravo,2020,September,20.0,6940.0,347.0,5.710800e+04,-95.768293
4,Bashirian-Kassulke,Hospital,Government,Böblingen,Feruprazole,Mood Stabilizers,Charlie,2020,December,565.0,64975.0,115.0,7.863833e+04,836.239193
5,Bashirian-Kassulke,Hospital,Government,Crailsheim,Symbitrim,Analgesics,Bravo,2019,August,14700.0,7879200.0,536.0,2.650372e+06,12026.510196
6,Bashirian-Kassulke,Hospital,Government,Dachau,Ketamara Evogel,Antipiretics,Bravo,2020,December,25.0,17600.0,704.0,2.653925e+06,-99.776627
7,Bashirian-Kassulke,Hospital,Government,Derne,Atrabicin Alkerotec,Antiseptics,Alfa,2019,November,100.0,24900.0,249.0,2.640567e+06,41.477273
8,Bashirian-Kassulke,Hospital,Government,Dreieich,Effidomide Evofribrate,Antiseptics,Alfa,2019,February,1440.0,881280.0,612.0,3.079267e+05,3439.277108
9,Bashirian-Kassulke,Hospital,Government,Elmshorn,Lovephilus,Analgesics,Charlie,2019,October,700.0,407400.0,582.0,4.378600e+05,-53.771786


In [9]:
features_cleaned[['total_sales', 'rolling_avg_sales_3m', 'sales_growth_pct']].describe()

,total_sales,rolling_avg_sales_3m,sales_growth_pct
count,2.536420e+05,2.536420e+05,2.536420e+05
mean,4.651827e+04,4.656789e+04,2.275587e+03
std,3.494946e+05,2.183254e+05,1.103242e+05
min,-4.161600e+06,-1.386920e+06,-9.025324e+05
25%,1.715000e+03,4.748333e+03,-8.234501e+01
50%,5.850000e+03,1.259400e+04,-2.009400e+00
75%,2.166000e+04,3.542650e+04,3.971059e+02
max,7.420560e+07,3.458677e+07,5.358808e+07


In [7]:
# Using IQR for outlier detection
Q1 = features_cleaned['total_sales'].quantile(0.25)
Q3 = features_cleaned['total_sales'].quantile(0.75)
IQR = Q3 - Q1
outlier = features_cleaned[(features_cleaned['total_sales'] < (Q1 - 1.5 * IQR)) | (features_cleaned['total_sales'] > Q3 + 1.5 * IQR)]
print(f"Number of outliers detected: {outlier.shape[0]}")

Number of outliers detected: 36097
